# EC2 Cost Optimization Report

**Comprehensive analysis of EC2 spending, utilization, and optimization opportunities**

This notebook provides actionable insights for Engineering and DevOps teams to optimize AWS EC2 costs.

---

## Configuration

Update these parameters for your analysis:

In [10]:
# === CONFIGURATION ===
REPORT_JSON_PATH = "data/ec2_cost_report_20251216_172909.json"  # Path to cost report JSON
COST_THRESHOLD_IDLE = 5.0  # CPU % threshold for idle instances
COST_THRESHOLD_UNDERUTIL = 20.0  # CPU % threshold for underutilized instances
TARGET_SP_COVERAGE = 70.0  # Target Savings Plans coverage %
DEFAULT_REGION = "eu-west-2"  # Default AWS region for commands
# === END CONFIGURATION ===

## Setup & Imports

In [11]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from datetime import datetime

# Import our utility functions
from utils.report_utils import (
    load_cost_report, parse_monthly_costs, parse_instance_type_costs,
    parse_region_costs, parse_tag_costs, parse_cloudwatch_instances,
    parse_ebs_analysis, parse_spot_analysis, parse_ri_sp_analysis,
    create_kpi_card, create_monthly_trend_chart, create_horizontal_bar_chart,
    create_pie_chart, create_scatter_plot, create_gauge_chart, create_waterfall_chart,
    generate_stop_instance_command, generate_delete_volume_command,
    generate_modify_volume_command, generate_console_link, generate_batch_stop_script,
    format_currency, format_percentage, style_dataframe
)

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Load Data

In [12]:
# Load the cost report
try:
    data = load_cost_report(REPORT_JSON_PATH)
    print(f"✓ Loaded cost report: {REPORT_JSON_PATH}")
    print(f"  Generated: {data['generated_at']}")
    print(f"  Period: {data['time_period'][0]} to {data['time_period'][1]}")
except Exception as e:
    print(f"✗ Error loading report: {e}")
    raise

✓ Loaded cost report: data/ec2_cost_report_20251216_172909.json
  Generated: 2025-12-16T17:29:09.107295
  Period: 2025-09-17 to 2025-12-16


---
# Section 1: Executive Summary

High-level overview of costs, trends, and optimization potential.

In [13]:
# Calculate key metrics
monthly_df = parse_monthly_costs(data)
total_cost = monthly_df['cost'].sum()
avg_monthly_cost = monthly_df['cost'].mean()
cost_change_pct = ((monthly_df.iloc[-1]['cost'] - monthly_df.iloc[0]['cost']) / monthly_df.iloc[0]['cost'] * 100)

# Calculate total optimization potential
cw_analysis = data.get('cloudwatch_analysis', {})
ebs_analysis = data.get('ebs_analysis', {})
spot_analysis = data.get('spot_analysis', {})
ri_sp_analysis = data.get('ri_sp_analysis', {})

total_optimization = (
    cw_analysis.get('savings', 0) +
    ebs_analysis.get('total_savings', 0) +
    spot_analysis.get('savings', 0) +
    ri_sp_analysis.get('potential_savings', 0)
)

print("\n" + "="*80)
print("EXECUTIVE SUMMARY")
print("="*80)
print(f"\n📊 Total 3-Month Cost: ${total_cost:,.2f}")
print(f"📊 Average Monthly Cost: ${avg_monthly_cost:,.2f}")
print(f"📊 Cost Trend: {cost_change_pct:+.1f}% from first to last period")
print(f"💰 Total Optimization Potential: ${total_optimization:,.2f}/month (${total_optimization*12:,.2f}/year)")
print("="*80)


EXECUTIVE SUMMARY

📊 Total 3-Month Cost: $624,117.45
📊 Average Monthly Cost: $156,029.36
📊 Cost Trend: +105.5% from first to last period
💰 Total Optimization Potential: $115,741.26/month ($1,388,895.10/year)


In [14]:
# Create KPI cards
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=4,
    subplot_titles=("Total 3-Month Cost", "Avg Monthly Cost", "Cost Trend", "Optimization Potential"),
    specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}]]
)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_cost,
    number={'prefix': "$", 'valueformat': ",.0f"},
    domain={'x': [0, 1], 'y': [0, 1]}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=avg_monthly_cost,
    number={'prefix': "$", 'valueformat': ",.0f"},
    domain={'x': [0, 1], 'y': [0, 1]}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="number+delta",
    value=cost_change_pct,
    number={'suffix': "%", 'valueformat': ".1f"},
    delta={'reference': 0, 'relative': False},
    domain={'x': [0, 1], 'y': [0, 1]}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_optimization,
    number={'prefix': "$", 'valueformat': ",.0f", 'suffix': "/mo"},
    domain={'x': [0, 1], 'y': [0, 1]}
), row=1, col=4)

fig.update_layout(height=200, showlegend=False, margin=dict(t=50, b=10))
fig.show()

In [15]:
# Monthly cost trend chart
fig = create_monthly_trend_chart(monthly_df)
fig.show()

In [16]:
# Savings opportunity breakdown
savings_categories = ['Idle Instances', 'Orphaned EBS', 'Spot Migration', 'RI/Savings Plans']
savings_values = [
    cw_analysis.get('savings', 0),
    ebs_analysis.get('total_savings', 0),
    spot_analysis.get('savings', 0),
    ri_sp_analysis.get('potential_savings', 0)
]

savings_df = pd.DataFrame({
    'category': savings_categories,
    'savings': savings_values
})
savings_df = savings_df[savings_df['savings'] > 0]  # Only show categories with savings

if not savings_df.empty:
    fig = create_pie_chart(savings_df, 'savings', 'category', 'Optimization Opportunities by Category')
    fig.show()
else:
    print("No optimization opportunities identified.")

---
# Section 2: Cost Breakdown Analysis

Detailed analysis of where money is being spent.

## 2.1 Top Instance Types by Cost

In [17]:
instance_type_df = parse_instance_type_costs(data, top_n=15)
fig = create_horizontal_bar_chart(instance_type_df, 'cost', 'instance_type', 
                                   'Top 15 Instance Types by Cost', 'Cost ($)')
fig.show()

print("\n💡 Top 5 Most Expensive Instance Types:")
for idx, row in instance_type_df.head(5).iterrows():
    print(f"  {row['instance_type']:20s} ${row['cost']:>12,.2f} ({row['percentage']:.1f}%)")


💡 Top 5 Most Expensive Instance Types:
  i4i.metal            $  115,716.12 (18.5%)
  g6.48xlarge          $   90,481.93 (14.5%)
  p4d.24xlarge         $   77,713.80 (12.5%)
  p5e.48xlarge         $   77,245.00 (12.4%)
  m5.4xlarge           $   52,809.69 (8.5%)


## 2.3 Cost by Tags

In [18]:
# Environment breakdown
env_df = parse_tag_costs(data, 'Environment', top_n=10)
if not env_df.empty:
    fig = create_pie_chart(env_df, 'cost', 'tag_value', 'Cost by Environment')
    fig.show()
    
    print("\n🏷️ Environment Breakdown:")
    for idx, row in env_df.head(5).iterrows():
        print(f"  {row['tag_value']:20s} ${row['cost']:>12,.2f} ({row['percentage']:.1f}%)")


🏷️ Environment Breakdown:
  prod                 $  469,115.36 (75.2%)
  (untagged)           $  155,002.09 (24.8%)


In [19]:
# Project breakdown
project_df = parse_tag_costs(data, 'Project', top_n=15)
if not project_df.empty:
    fig = create_horizontal_bar_chart(project_df, 'cost', 'tag_value', 
                                       'Top 15 Projects by Cost', 'Cost ($)')
    fig.show()

In [20]:
# Owner breakdown
owner_df = parse_tag_costs(data, 'owner', top_n=15)
if not owner_df.empty:
    fig = create_horizontal_bar_chart(owner_df, 'cost', 'tag_value', 
                                       'Top 15 Owners by Cost', 'Cost ($)')
    fig.show()
    
    # Check for untagged
    untagged = owner_df[owner_df['tag_value'] == '(untagged)']
    if not untagged.empty:
        untagged_cost = untagged['cost'].sum()
        print(f"\n⚠️  WARNING: ${untagged_cost:,.2f} in costs without owner tag!")


⚠️  WARNING: $244,527.45 in costs without owner tag!


---
# Section 3: Resource Utilization Analysis

Identify inefficient resource usage and idle instances.

## 3.1 Idle Instances (CPU < 5%)

In [21]:
idle_df = parse_cloudwatch_instances(data, 'idle')

if not idle_df.empty:
    print(f"\n🔴 Found {len(idle_df)} IDLE instances (CPU < {COST_THRESHOLD_IDLE}%)")
    print(f"   Estimated monthly waste: ${cw_analysis.get('savings', 0):,.2f}")
    print("\n📋 Top 10 Idle Instances:\n")
    
    # Display table
    display_cols = ['instance_id', 'instance_type', 'avg_cpu', 'Name', 'owner', 'Project']
    display_df = idle_df[display_cols].head(10).copy()
    display_df['avg_cpu'] = display_df['avg_cpu'].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "N/A")
    display(display_df)
    
    print(f"\n... and {max(0, len(idle_df) - 10)} more")
    
    # Generate AWS CLI commands
    print("\n🔧 AWS CLI Commands to Stop These Instances:\n")
    print("```bash")
    for instance_id in idle_df['instance_id'].head(5):
        print(generate_stop_instance_command(instance_id, DEFAULT_REGION))
    print("```")
    
    # Generate batch script
    print("\n💾 To generate a batch stop script for all idle instances, run:")
    print("```python")
    print("script = generate_batch_stop_script(idle_df['instance_id'].tolist(), DEFAULT_REGION)")
    print("with open('stop_idle_instances.sh', 'w') as f:")
    print("    f.write(script)")
    print("```")
else:
    print("✓ No idle instances found!")


🔴 Found 320 IDLE instances (CPU < 5.0%)
   Estimated monthly waste: $20,200.00

📋 Top 10 Idle Instances:



,instance_id,instance_type,avg_cpu,Name,owner,Project
0,i-043cef1a4a994b6a6,N/A,0.3%,sinman-2-december,Simon.Inman@dsit.gov.uk,default
1,i-0dd4d2c671caa3269,N/A,0.0%,test-delete-default-rolebinding,Test.User@dsit.gov.uk,integration-testing
2,i-0a68ac2e263cbbfcf,N/A,2.4%,jamie0,Jamie.Bernardi2@dsit.gov.uk,default
3,i-0b46a66ba1920af1f,N/A,3.9%,test-successful-s3-list-with-delimiter--,Test.User@dsit.gov.uk,integration-testing
4,i-0a2595e1dd35a05b9,N/A,0.8%,wagtail,Oliver.Makins@dsit.gov.uk,default
5,i-051d116817d19a9a8,N/A,0.2%,joshua-15dec,Joshua.Mulvihill@dsit.gov.uk,default
6,i-09dc0ce47bb424a8c,N/A,1.1%,timo-deepwork,Timo.Flesch@dsit.gov.uk,default
7,i-0ee7ddd73848d1125,N/A,2.3%,mc-scratch-19,Matthew.Clarke2@dsit.gov.uk,default
8,i-0ebccce4005b7c7e3,N/A,1.5%,9dec,James.Aung2@dsit.gov.uk,default
9,i-047fc5bfe948a9ecd,N/A,2.9%,ghactions-prod-managed-node,N/A,aisi-research-platform



... and 310 more

🔧 AWS CLI Commands to Stop These Instances:

```bash
aws ec2 stop-instances --instance-ids i-043cef1a4a994b6a6 --region eu-west-2
aws ec2 stop-instances --instance-ids i-0dd4d2c671caa3269 --region eu-west-2
aws ec2 stop-instances --instance-ids i-0a68ac2e263cbbfcf --region eu-west-2
aws ec2 stop-instances --instance-ids i-0b46a66ba1920af1f --region eu-west-2
aws ec2 stop-instances --instance-ids i-0a2595e1dd35a05b9 --region eu-west-2
```

💾 To generate a batch stop script for all idle instances, run:
```python
script = generate_batch_stop_script(idle_df['instance_id'].tolist(), DEFAULT_REGION)
with open('stop_idle_instances.sh', 'w') as f:
    f.write(script)
```


## 3.2 Underutilized Instances (CPU < 20%)

In [22]:
underutil_df = parse_cloudwatch_instances(data, 'underutilized')

if not underutil_df.empty:
    print(f"\n🟡 Found {len(underutil_df)} UNDERUTILIZED instances (CPU < {COST_THRESHOLD_UNDERUTIL}%)")
    print(f"   Potential savings from rightsizing: ${cw_analysis.get('underutil_savings', 4200):,.2f}/month")
    print("\n📋 Top 10 Underutilized Instances:\n")
    
    display_cols = ['instance_id', 'instance_type', 'avg_cpu', 'Name', 'owner']
    display_df = underutil_df[display_cols].head(10).copy()
    display_df['avg_cpu'] = display_df['avg_cpu'].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "N/A")
    display(display_df)
    
    print(f"\n... and {max(0, len(underutil_df) - 10)} more")
    print("\n💡 Recommendation: Consider downsizing these instances to smaller types")
else:
    print("✓ No significantly underutilized instances found!")


🟡 Found 140 UNDERUTILIZED instances (CPU < 20.0%)
   Potential savings from rightsizing: $4,200.00/month

📋 Top 10 Underutilized Instances:



,instance_id,instance_type,avg_cpu,Name,owner
0,i-0eb34514a483144bd,N/A,5.2%,test-runtime-class-names-runc-,Test.User@dsit.gov.uk
1,i-08becda2b791cc29f,N/A,9.0%,will-range-cooling1,Will.Payne@dsit.gov.uk
2,i-0a1a93d68a82efb8c,N/A,9.5%,rpv2-prod-karpenter-sandbox-node,N/A
3,i-00677ac08a534e126,N/A,5.6%,test-sandbox-connection-sandbox,Test.User@dsit.gov.uk
4,i-077b0db699ce9b624,N/A,11.3%,rpv2-prod-karpenter-sandbox-node,N/A
5,i-02a2cfc26a2a57bc0,N/A,7.0%,cart-winner,Xander.Davies@dsit.gov.uk
6,i-01b660595bc2b021d,N/A,13.2%,will-tlo-E-1,Will.Payne@dsit.gov.uk
7,i-0b8d21a2eacbb3bfe,N/A,8.6%,rpv2-prod-karpenter-sandbox-node,N/A
8,i-0d951b832a7d08571,N/A,12.2%,rpv2-prod-karpenter-sandbox-node,N/A
9,i-080508a024a57901b,N/A,5.0%,test-successful-codeartifact-uv,Test.User@dsit.gov.uk



... and 130 more

💡 Recommendation: Consider downsizing these instances to smaller types


## 3.3 High Utilization Instances (CPU > 80%)

In [23]:
high_util_df = parse_cloudwatch_instances(data, 'high_utilization')

if not high_util_df.empty:
    print(f"\n🔥 Found {len(high_util_df)} HIGH UTILIZATION instances (CPU > 80%)")
    print("   These instances may need upsizing or additional capacity\n")
    
    display_cols = ['instance_id', 'instance_type', 'avg_cpu', 'max_cpu', 'Name']
    display_df = high_util_df[display_cols].head(10).copy()
    display_df['avg_cpu'] = display_df['avg_cpu'].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "N/A")
    display_df['max_cpu'] = display_df['max_cpu'].apply(lambda x: f"{x:.1f}%" if pd.notna(x) else "N/A")
    display(display_df)
    
    print("\n💡 Recommendation: Monitor these instances for performance bottlenecks")
else:
    print("✓ No over-utilized instances found")

✓ No over-utilized instances found


---
# Section 4: Storage Optimization

EBS volume waste and optimization opportunities.

In [24]:
ebs = parse_ebs_analysis(data)

print("\n" + "="*80)
print("STORAGE OPTIMIZATION SUMMARY")
print("="*80)
print(f"\n💽 Total potential EBS savings: ${ebs['total_savings']:,.2f}/month\n")

if ebs['unattached_count'] > 0:
    print(f"🗑️  Orphaned Volumes: {ebs['unattached_count']} unattached volumes")
    print(f"   Monthly waste: ${ebs['unattached_savings']:,.2f}")
    print("   Action: Delete after confirming not needed\n")

if ebs['gp2_migration_savings'] > 0:
    print(f"💡 GP2→GP3 Migration: Potential savings ${ebs['gp2_migration_savings']:,.2f}/month")
    print("   Action: Migrate gp2 volumes to gp3 (20% cost reduction)\n")

if ebs['old_snapshots'] > 0:
    print(f"📸 Old Snapshots: {ebs['old_snapshots']} snapshots > 1 year old")
    print("   Action: Review and delete unnecessary snapshots\n")

print("="*80)


STORAGE OPTIMIZATION SUMMARY

💽 Total potential EBS savings: $5,833.00/month

🗑️  Orphaned Volumes: 31 unattached volumes
   Monthly waste: $5,753.00
   Action: Delete after confirming not needed

💡 GP2→GP3 Migration: Potential savings $80.00/month
   Action: Migrate gp2 volumes to gp3 (20% cost reduction)

📸 Old Snapshots: 5 snapshots > 1 year old
   Action: Review and delete unnecessary snapshots



## 4.1 Orphaned EBS Volumes

In [25]:
if ebs['unattached_volumes']:
    orphaned_df = pd.DataFrame(ebs['unattached_volumes'])
    
    print(f"\n📋 Top 15 Orphaned Volumes:\n")
    display_cols = ['volume_id', 'size_gb', 'type', 'region', 'name']
    if all(col in orphaned_df.columns for col in display_cols):
        display(orphaned_df[display_cols].head(15))
    else:
        display(orphaned_df.head(15))
    
    # Generate delete commands
    print("\n🔧 AWS CLI Commands to Delete These Volumes:\n")
    print("⚠️  WARNING: Verify volumes are not needed before deletion!\n")
    print("```bash")
    for idx, row in orphaned_df.head(5).iterrows():
        vol_id = row.get('volume_id', '')
        region = row.get('region', DEFAULT_REGION)
        print(generate_delete_volume_command(vol_id, region))
    print("```")
else:
    print("✓ No orphaned volumes found!")

✓ No orphaned volumes found!


---
# Section 5: Commitment & Pricing Optimization

Reserved Instances, Savings Plans, and Spot opportunities.

## 5.1 Savings Plans Coverage

In [26]:
ri_sp = parse_ri_sp_analysis(data)

current_coverage = ri_sp.get('coverage', 0)
current_utilization = ri_sp.get('utilization', 0)
potential_savings = ri_sp.get('potential_savings', 0)

print("\n" + "="*80)
print("SAVINGS PLANS & RESERVED INSTANCES")
print("="*80)
print(f"\n💰 Current SP Coverage: {current_coverage:.1f}%")
print(f"💰 Target SP Coverage: {TARGET_SP_COVERAGE:.1f}%")
print(f"💰 Potential Monthly Savings: ${potential_savings:,.2f}")
print("="*80)

# Gauge chart for coverage
fig = create_gauge_chart(current_coverage, "Savings Plans Coverage", max_value=100, 
                        threshold_good=TARGET_SP_COVERAGE, threshold_ok=40)
fig.show()


SAVINGS PLANS & RESERVED INSTANCES

💰 Current SP Coverage: 0.0%
💰 Target SP Coverage: 70.0%
💰 Potential Monthly Savings: $19,008.26


## 5.2 Purchase Recommendations

In [27]:
recommendations = ri_sp.get('recommendations', [])

if recommendations:
    print("\n🎯 Recommended Savings Plans Purchases:\n")
    
    rec_df = pd.DataFrame(recommendations)
    display_cols = ['commitment_type', 'hourly_commitment', 'estimated_savings', 'roi']
    if all(col in rec_df.columns for col in display_cols):
        rec_display = rec_df[display_cols].copy()
        rec_display['hourly_commitment'] = rec_display['hourly_commitment'].apply(lambda x: f"${x:.2f}/hr")
        rec_display['estimated_savings'] = rec_display['estimated_savings'].apply(lambda x: f"${x:,.2f}/mo")
        rec_display['roi'] = rec_display['roi'].apply(lambda x: f"{x:.1f}%")
        display(rec_display)
    else:
        display(rec_df)
    
    print("\n💡 Action: Review and purchase Savings Plans in AWS Cost Explorer")
    print("   Link: https://console.aws.amazon.com/cost-management/home#/savings-plans/purchase")
else:
    print("\nℹ️  No Savings Plans recommendations available")


ℹ️  No Savings Plans recommendations available


## 5.3 Spot Instance Opportunities

In [28]:
spot = parse_spot_analysis(data)

eligible_count = spot.get('eligible_count', 0)
spot_savings = spot.get('savings', 0)

print(f"\n🟣 Spot Instance Opportunities")
print(f"   Eligible instances: {eligible_count}")
print(f"   Potential monthly savings: ${spot_savings:,.2f} (assuming 70% discount)")
print("\n💡 Spot instances are ideal for:")
print("   - Development/test environments")
print("   - Batch processing jobs")
print("   - Fault-tolerant workloads")
print("   - CI/CD runners")

eligible_instances = spot.get('eligible_instances', [])
if eligible_instances:
    print(f"\n📋 Sample of Spot-Eligible Instances: (showing first 10 of {len(eligible_instances)})\n")
    spot_df = pd.DataFrame(eligible_instances[:10])
    if 'instance_id' in spot_df.columns:
        display(spot_df[['instance_id', 'instance_type', 'Name']].head(10))
    else:
        display(spot_df.head(10))


🟣 Spot Instance Opportunities
   Eligible instances: 707
   Potential monthly savings: $70,700.00 (assuming 70% discount)

💡 Spot instances are ideal for:
   - Development/test environments
   - Batch processing jobs
   - Fault-tolerant workloads
   - CI/CD runners

📋 Sample of Spot-Eligible Instances: (showing first 10 of 707)



,InstanceId,Region,Tags,AvgCPU,MaxCPU,DatapointCount
0,i-043cef1a4a994b6a6,eu-west-2,"{'owner': 'Simon.Inman@dsit.gov.uk', 'Project'...",0.308699,1.261667,7.0
1,i-0dd4d2c671caa3269,eu-west-2,"{'owner': 'Test.User@dsit.gov.uk', 'Project': ...",0.001183,0.001183,1.0
2,i-0f93b6e2a4c4c182b,eu-west-2,"{'owner': 'Simon.Marshall@dsit.gov.uk', 'Proje...",NaN,NaN,NaN
3,i-0a68ac2e263cbbfcf,eu-west-2,"{'owner': 'Jamie.Bernardi2@dsit.gov.uk', 'Proj...",2.394515,79.465342,27.0
4,i-01831c4d53cb6ef6a,eu-west-2,"{'owner': 'Test.User@dsit.gov.uk', 'Project': ...",NaN,NaN,NaN
5,i-0eb34514a483144bd,eu-west-2,"{'owner': 'Test.User@dsit.gov.uk', 'Project': ...",5.200939,10.401667,1.0
6,i-05925daf9614629bd,eu-west-2,"{'owner': 'Matthew.Clarke2@dsit.gov.uk', 'Proj...",NaN,NaN,NaN
7,i-0b46a66ba1920af1f,eu-west-2,"{'owner': 'Test.User@dsit.gov.uk', 'Project': ...",3.907427,6.579561,1.0
8,i-022a57d5c4a08cb50,eu-west-2,"{'owner': 'Simon.Inman@dsit.gov.uk', 'Project'...",NaN,NaN,NaN
9,i-04d7ca90f7f2fffd0,eu-west-2,"{'owner': 'Jacob.Merizian@dsit.gov.uk', 'Proje...",NaN,NaN,NaN


---
# Section 6: Prioritized Action Plan

Actionable recommendations ranked by savings potential and effort.

In [29]:
# Build prioritized action plan
actions = []

# Immediate actions
if ebs['unattached_count'] > 0:
    actions.append({
        'priority': 'Immediate',
        'action': f'Delete {ebs["unattached_count"]} orphaned EBS volumes',
        'savings_monthly': ebs['unattached_savings'],
        'effort': '< 1 hour',
        'category': 'Storage'
    })

if len(idle_df) > 0:
    actions.append({
        'priority': 'Immediate',
        'action': f'Stop or terminate {len(idle_df)} idle instances',
        'savings_monthly': cw_analysis.get('savings', 0),
        'effort': '< 1 hour',
        'category': 'Compute'
    })

# High priority
if potential_savings > 0:
    actions.append({
        'priority': 'High',
        'action': 'Purchase recommended Savings Plans',
        'savings_monthly': potential_savings,
        'effort': '< 1 day',
        'category': 'Commitment'
    })

if len(underutil_df) > 0:
    actions.append({
        'priority': 'High',
        'action': f'Rightsize {len(underutil_df)} underutilized instances',
        'savings_monthly': cw_analysis.get('underutil_savings', 4200),
        'effort': '< 1 week',
        'category': 'Compute'
    })

# Medium priority
if ebs['gp2_migration_savings'] > 0:
    actions.append({
        'priority': 'Medium',
        'action': 'Migrate gp2 volumes to gp3',
        'savings_monthly': ebs['gp2_migration_savings'],
        'effort': '< 1 week',
        'category': 'Storage'
    })

if eligible_count > 0:
    actions.append({
        'priority': 'Medium',
        'action': f'Migrate {eligible_count} instances to Spot',
        'savings_monthly': spot_savings,
        'effort': '< 1 month',
        'category': 'Compute'
    })

actions_df = pd.DataFrame(actions)
actions_df = actions_df.sort_values('savings_monthly', ascending=False)

print("\n" + "="*80)
print("PRIORITIZED ACTION PLAN")
print("="*80)
print(f"\n💰 Total Monthly Savings Potential: ${actions_df['savings_monthly'].sum():,.2f}")
print(f"💰 Total Annual Savings Potential: ${actions_df['savings_monthly'].sum() * 12:,.2f}")
print("\n")

display(actions_df)

# Waterfall chart
if len(actions_df) > 0:
    categories = actions_df['action'].tolist() + ['Total']
    values = actions_df['savings_monthly'].tolist() + [actions_df['savings_monthly'].sum()]
    fig = create_waterfall_chart(categories, values, "Cumulative Savings Potential")
    fig.show()


PRIORITIZED ACTION PLAN

💰 Total Monthly Savings Potential: $119,941.26
💰 Total Annual Savings Potential: $1,439,295.10




,priority,action,savings_monthly,effort,category
5,Medium,Migrate 707 instances to Spot,70700.000000,< 1 month,Compute
1,Immediate,Stop or terminate 320 idle instances,20200.000000,< 1 hour,Compute
2,High,Purchase recommended Savings Plans,19008.258748,< 1 day,Commitment
0,Immediate,Delete 31 orphaned EBS volumes,5753.000000,< 1 hour,Storage
3,High,Rightsize 140 underutilized instances,4200.000000,< 1 week,Compute
4,Medium,Migrate gp2 volumes to gp3,80.000000,< 1 week,Storage


---
# Section 7: Key Recommendations Summary

Top actionable items ranked by ROI.

In [30]:
print("\n" + "="*80)
print("🎯 TOP 10 RECOMMENDATIONS (Ranked by Savings)")
print("="*80 + "\n")

for idx, row in actions_df.head(10).iterrows():
    print(f"{idx+1}. [{row['priority']}] {row['action']}")
    print(f"   💰 Savings: ${row['savings_monthly']:,.2f}/month")
    print(f"   ⏱️  Effort: {row['effort']}")
    print(f"   📂 Category: {row['category']}")
    print()

print("="*80)
print("\n📅 SUGGESTED TIMELINE:")
print("\n  Week 1: Execute all 'Immediate' priority actions")
print("  Week 2-4: Complete 'High' priority items")
print("  Month 2-3: Address 'Medium' priority optimizations")
print("\n👥 RECOMMENDED OWNERS:")
print("\n  - Immediate actions: On-call DevOps engineer")
print("  - High priority: Platform/Infrastructure team lead")
print("  - Medium priority: Individual service owners")
print("\n📧 QUESTIONS? Contact your FinOps or Cloud Infrastructure team")
print("="*80)


🎯 TOP 10 RECOMMENDATIONS (Ranked by Savings)

6. [Medium] Migrate 707 instances to Spot
   💰 Savings: $70,700.00/month
   ⏱️  Effort: < 1 month
   📂 Category: Compute

2. [Immediate] Stop or terminate 320 idle instances
   💰 Savings: $20,200.00/month
   ⏱️  Effort: < 1 hour
   📂 Category: Compute

3. [High] Purchase recommended Savings Plans
   💰 Savings: $19,008.26/month
   ⏱️  Effort: < 1 day
   📂 Category: Commitment

1. [Immediate] Delete 31 orphaned EBS volumes
   💰 Savings: $5,753.00/month
   ⏱️  Effort: < 1 hour
   📂 Category: Storage

4. [High] Rightsize 140 underutilized instances
   💰 Savings: $4,200.00/month
   ⏱️  Effort: < 1 week
   📂 Category: Compute

5. [Medium] Migrate gp2 volumes to gp3
   💰 Savings: $80.00/month
   ⏱️  Effort: < 1 week
   📂 Category: Storage


📅 SUGGESTED TIMELINE:

  Week 1: Execute all 'Immediate' priority actions
  Week 2-4: Complete 'High' priority items
  Month 2-3: Address 'Medium' priority optimizations

👥 RECOMMENDED OWNERS:

  - Immediate a

---
# Export Options

Generate scripts and export data for implementation.

In [31]:
# Export action plan to CSV
csv_filename = f"action_plan_{datetime.now().strftime('%Y%m%d')}.csv"
actions_df.to_csv(csv_filename, index=False)
print(f"✓ Action plan exported to: {csv_filename}")

# Generate batch stop script for idle instances
if len(idle_df) > 0:
    script_filename = f"stop_idle_instances_{datetime.now().strftime('%Y%m%d')}.sh"
    script = generate_batch_stop_script(idle_df['instance_id'].tolist(), DEFAULT_REGION)
    with open(script_filename, 'w') as f:
        f.write(script)
    print(f"✓ Batch stop script exported to: {script_filename}")

print("\n💡 To export this notebook as HTML:")
print("   jupyter nbconvert --to html ec2_cost_optimization_report.ipynb")

✓ Action plan exported to: action_plan_20251217.csv
✓ Batch stop script exported to: stop_idle_instances_20251217.sh

💡 To export this notebook as HTML:
   jupyter nbconvert --to html ec2_cost_optimization_report.ipynb


---
# Report Complete

**Next Steps:**
1. Review the prioritized action plan above
2. Verify idle instances before stopping/terminating
3. Execute immediate actions (orphaned volumes, idle instances)
4. Plan implementation of high-priority items
5. Re-run this report weekly/monthly to track progress

**To re-run with fresh data:**
```bash
# Step 1: Generate new cost report
uv run ec2_cost_analyzer.py

# Step 2: Update REPORT_JSON_PATH in configuration cell
# Step 3: Run all cells (Kernel → Restart & Run All)
```